In [1]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from api_keys import g_key
import gmaps
import requests
import json

In [2]:
gmaps.configure(api_key=g_key)

In [3]:
df = pd.read_csv("booth_cities.csv")
df.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Khani,41.96,42.96,64.40,60.37,63,1015,8.05,8.05
1,Marang,5.21,103.21,75.09,82.56,94,1010,3.36,3.36
2,Labuhan,-6.88,112.21,81.72,87.93,78,1012,6.51,6.51
3,Ushuaia,-54.80,-68.30,57.20,51.21,54,998,6.93,6.93
4,Lebu,-37.62,-73.65,50.86,46.35,78,1024,5.66,5.66


In [4]:
locations = df[["Latitude", "Longitude"]]
humids = df["Humidity"].astype(float)
locations

,Latitude,Longitude
0,41.96,42.96
1,5.21,103.21
2,-6.88,112.21
3,-54.80,-68.30
4,-37.62,-73.65
...,...,...
556,44.62,-65.77
557,11.28,-15.83
558,-0.58,117.09
559,29.03,21.55


In [7]:
ideal = (df.Temperature > 70) & (df.Temperature < 80) & (df.Cloudiness < 2) & (df["Wind Speed"] < 10)
df_drop = df.loc[ideal].reset_index(drop = True)
coordinates = df[["Latitude", "Longitude"]]
humids = df.Humidity
coordinates_ideal = df_drop[["Latitude", "Longitude"]]
df_drop.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Wattegama,6.80,81.48,74.77,82.62,90,1010,1.12,1.12
1,Durban,-29.86,31.03,70.21,75.54,88,1013,1.12,1.12
2,Omboué,-1.57,9.26,75.36,83.35,91,1011,1.72,1.72
3,Aketi,2.74,23.78,71.46,78.48,96,1009,1.32,1.32
4,Rafaela,-31.25,-61.49,70.81,67.37,32,1014,1.99,1.99


In [8]:
figure_layout = {
    'width': '800px',
    'height': '1100px',
    'border': '5px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")
heat = gmaps.heatmap_layer(coordinates, weights = humids, dissipating=False, max_intensity=700)
fig.add_layer(heat)
fig

Figure(layout=FigureLayout(border='5px solid black', height='1100px', margin='0 auto 0 auto', padding='1px', w…

In [10]:
ideal = (df.Temperature > 70) & (df.Temperature < 80) & (df.Cloudiness < 2) & (df["Wind Speed"] < 10)
df_drop = df.loc[ideal].reset_index(drop = True)
df_drop.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Wattegama,6.80,81.48,74.77,82.62,90,1010,1.12,1.12
1,Durban,-29.86,31.03,70.21,75.54,88,1013,1.12,1.12
2,Omboué,-1.57,9.26,75.36,83.35,91,1011,1.72,1.72
3,Aketi,2.74,23.78,71.46,78.48,96,1009,1.32,1.32
4,Rafaela,-31.25,-61.49,70.81,67.37,32,1014,1.99,1.99


In [11]:
df_drop.City.values

array(['Wattegama', 'Durban', 'Omboué', 'Aketi', 'Rafaela', 'Coroatá',
       'Saint George', 'Mangai', 'Lastoursville', 'Rantauprapat',
       'Chum Phae', 'Champasak', 'Uruaçu', 'Puerto Lleras', 'Moanda'],
      dtype=object)

In [12]:
#4 resulting cities labeled
figure_layout = {
    'width': '800px',
    'height': '1100px',
    'border': '5px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")
markers = gmaps.marker_layer(coordinates_ideal, info_box_content=df_drop.City)
fig.add_layer(markers)
heat = gmaps.heatmap_layer(coordinates, weights = humids, dissipating=False, max_intensity=700)
fig.add_layer(heat)
fig

Figure(layout=FigureLayout(border='5px solid black', height='1100px', margin='0 auto 0 auto', padding='1px', w…

In [13]:
hotel_df = df_drop[["City", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Latitude,Longitude,Hotel Name
0,Wattegama,6.80,81.48,
1,Durban,-29.86,31.03,
2,Omboué,-1.57,9.26,
3,Aketi,2.74,23.78,
4,Rafaela,-31.25,-61.49,


In [14]:
names = []
addresses = []
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
}

for index, row in df_drop.iterrows():
    
    location = f"{row.Latitude}, {row.Longitude}"
    params['location'] = location
    
    #Request
    print(f"Retrieving Results for Index {index}: {row.City}.")
    response = requests.get(base_url, params=params).json()
    
    # Results
    results = response['results']
    
    #Showing names of closest hotel
    try:
        print(f"Closest Hotel is {results[0]['name']}.")
              
        name = results[0]['name']
        address = results[0]['vicinity']
              
        names.append(name)
        addresses.append(address)
              
        
    except (KeyError, IndexError):
        print("Missing field...skipping.")
        names.append(np.nan)
        addresses.append(np.nan)
        
    print("------------")
              
df_drop["hotel_name"] = names
df_drop["address"] = addresses

Retrieving Results for Index 0: Wattegama.
Closest Hotel is Sugar Factory.
------------
Retrieving Results for Index 1: Durban.
Closest Hotel is Turners Warehouse (Pty) LTD.
------------
Retrieving Results for Index 2: Omboué.
Closest Hotel is Medical Center Onanga Cécile.
------------
Retrieving Results for Index 3: Aketi.
Closest Hotel is Assemblée Chretienne du Message du Temps de la Fin Tabernacle d'AKETI.
------------
Retrieving Results for Index 4: Rafaela.
Closest Hotel is Ventas Y Servicios SA.
------------
Retrieving Results for Index 5: Coroatá.
Closest Hotel is Comercial Santos.
------------
Retrieving Results for Index 6: Saint George.
Closest Hotel is Ronald Wilkinson Artist.
------------
Retrieving Results for Index 7: Mangai.
Closest Hotel is Mangai Ste Famille Centre de Santé.
------------
Retrieving Results for Index 8: Lastoursville.
Closest Hotel is Lastoursville Grand Marché.
------------
Retrieving Results for Index 9: Rantauprapat.
Closest Hotel is Mie Pansit Pane

In [21]:
df_drop

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness,hotel_name,address
0,Wattegama,6.80,81.48,74.77,82.62,90,1010,1.12,1.12,Sugar Factory,"sugar factory in, Wattegama"
1,Durban,-29.86,31.03,70.21,75.54,88,1013,1.12,1.12,Turners Warehouse (Pty) LTD,"10 Jonsson Lane, Durban Central, Durban"
2,Omboué,-1.57,9.26,75.36,83.35,91,1011,1.72,1.72,Medical Center Onanga Cécile,Omboue
3,Aketi,2.74,23.78,71.46,78.48,96,1009,1.32,1.32,Assemblée Chretienne du Message du Temps de la...,Aketi
4,Rafaela,-31.25,-61.49,70.81,67.37,32,1014,1.99,1.99,Ventas Y Servicios SA,"Lavalle 311, Rafaela"
5,Coroatá,-4.13,-44.12,75.52,84.02,93,1011,1.63,1.63,Comercial Santos,"502, Rua do Fio, 412, Coroatá"
6,Saint George,37.10,-113.58,78.80,75.11,19,1023,0.49,0.49,Ronald Wilkinson Artist,"524 South 200 East #4, St. George"
7,Mangai,-4.05,19.53,70.29,77.20,98,1010,1.01,1.01,Mangai Ste Famille Centre de Santé,Kwilu
8,Lastoursville,-0.81,12.73,72.82,80.29,95,1012,1.48,1.48,Lastoursville Grand Marché,Lastoursville
9,Rantauprapat,2.10,99.83,75.00,81.97,86,1012,1.66,1.66,Mie Pansit Pane,"Jalan Pane No.3, Kartini"


In [39]:
label_info = []
for indx, row in df_drop.iterrows():
    info = f"City: {row.City}<br>-----------<br>Hotel: {row['hotel_name']}<br>-----------<br>Address: {row.address}"
    label_info.append(info)

In [40]:
# Plotting Hotels onto heatmap
figure_layout = {
    'width': '800px',
    'height': '1100px',
    'border': '5px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

markers = gmaps.marker_layer(coordinates_ideal, info_box_content=label_info)
fig.add_layer(markers)
heat = gmaps.heatmap_layer(coordinates, weights = humids, dissipating=False, max_intensity=700)
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='5px solid black', height='1100px', margin='0 auto 0 auto', padding='1px', w…